<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/16_Chainladder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chainladder clásico con pandas y numpy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
df = pd.read_csv('cdiae/data/01_raw/data_claims.csv', sep=';')
df.head()

Una vez importados y formateados los datos de fecha, podemos crear nuevos campos con el año del siniestro y el año del pago (es decir, Accident_Year) o, en este caso, modificar los existentes:

In [ ]:
df['Accident_Date'] = pd.to_datetime(df['Accident_Date'])
df['Payment_Date'] = pd.to_datetime(df['Payment_Date'])

In [ ]:
df['Accident_Date'] = df['Accident_Date'].dt.to_period(freq='Y')
df['Payment_Date'] = df['Payment_Date'].dt.to_period(freq='Y')

Calculamos un nuevo campo llamado ’Dev’ con el periodo de tiempo (o desarrollo) como la diferencia entre las dos fechas (en tipo entero):

In [ ]:
df['Dev'] = df['Payment_Date'].astype(int) - df['Accident_Date'].astype(int)

Agrupamos por año de siniestro y año de desarrollo usando groupby(), y sumamos los importes de los siniestros:

In [ ]:
df_agg = df.groupby(['Accident_Date','Dev']).agg({'Indemnity':'sum'}).reset_index()

Una vez agrupados, guardamos los datos en una tabla bidimensional (pivot table) basada en estas dos variables:

In [ ]:
triangle = df_agg.pivot(index='Accident_Date', columns='Dev', values='Indemnity')
triangle

El triángulo se calcula con los pagos acumulados a lo largo del tiempo. La función .cumsum() puede utilizarse para acumular los datos de un dataframe, aunque por defecto suma los datos de una columna, por lo que se debe especificar la suma por filas (axis=1).

In [ ]:
accumulate = triangle.cumsum(axis=1)
accumulate

Se calculan los patrones de pago o factores de desarrollo: cuotas incrementales, link ratios o **factores de desarrollo de siniestros (LDFs)**, y las cuotas acumuladas, ratios finales o **factores de desarrollo acumulados (CDFs)**.

In [ ]:
n = len(accumulate)-1

In [ ]:
LDFs = np.ones(n)
for i in range(n):
	SumDev = pd.Series(accumulate[i][:n-i]).sum(skipna=True)
	SumDevNext = pd.Series(accumulate[min(i+1,n)][:n-i]).sum(skipna=True)
	LDFs[i] = SumDevNext/SumDev

In [ ]:
CDFs = np.ones(n+1) #fix last value as 1.
for i in range(n):
	CDFs[i] = np.prod(LDFs[i:n])

Calcular el patrón de pagos con estos valores es sencillo. La función **.reciprocal()** devuelve el inverso del número (1/x). En la columna de tasas incrementales, los valores *NaN* se sustituyen por el valor de la columna de tasas acumuladas

In [ ]:
payment_pattern = pd.DataFrame(data=np.reciprocal(CDFs),columns=['Cum'])
payment_pattern['Incr'] = payment_pattern['Cum'].diff().fillna(payment_pattern['Cum'])

Graficamos:

In [ ]:
import matplotlib.pyplot as plt
payment_pattern.plot()
plt.show()

Multiplicamos los pagos por los ratios de desarrollo finales (hasta el último valor) para obtener el valor del último pago, es decir, al restar los pagos actuales obtenemos el valor de la reserva necesaria.

In [ ]:
diagonal = np.diag(np.flipud(accumulate))
payments = pd.Series(data=diagonal)
ultimate = np.dot(CDFs, payments)


Cálculo de la reserva:

In [ ]:
reserve = ultimate - np.sum(payments)
print(f'Reserva: {reserve:,.2f}')